In [34]:

from models.doc2vec_model import doc2VecModel
from models.classifier_model import classifierModel

import os
import logging
import inspect

import pandas as pd
from sklearn.model_selection import train_test_split

Homedir = os.getcwd()
logging.basicConfig(
    format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
base_file_path = inspect.getframeinfo(inspect.currentframe()).filename
project_dir_path = os.path.dirname(os.path.abspath(base_file_path))
data_path = os.path.join(project_dir_path, 'data')
default_classifier = os.path.join(
    project_dir_path, 'classifiers', 'logreg_model.pkl')
default_doc2vec = os.path.join(project_dir_path, 'classifiers', 'd2v.model')
default_dataset = os.path.join(data_path, 'dataset.csv')


class TextClassifier():

    def __init__(self):
        super().__init__()
        self.d2v = doc2VecModel()
        self.classifier = classifierModel()
        self.dataset = None
        self.dataset1 = None

    def read_data(self):
        #filename = os.path.join(data_path, filename)

        # dfdrugbank = pd.read_csv("C:/git/drugbank/data/drugbank.tsv", delimiter="\t")
        # dfprotein = pd.read_csv("C:/git/drugbank/data/proteins.tsv", delimiter="\t")
        # dfdrugbank['DrugBank_id'] = dfdrugbank['drugbank_id']
        # dfprotein['DrugBank_id'] = dfprotein['drugbank_id']
        # dfnodes = pd.read_csv("C:/git/BioNEV/data/DrugBank_DDI/node_list.txt", delimiter="	")
        # dfnodes['ID'] = dfnodes.index
        # dfdrugbank2 = dfdrugbank.merge(dfnodes, how="left", on="DrugBank_id")
        # dfdrugbank2 = dfdrugbank.merge(dfprotein, how="left", on="DrugBank_id")
        # dfdrugbank2.dropna(inplace=True)
        # from sklearn.preprocessing import LabelEncoder
        #
        # lb_make = LabelEncoder()
        # dfdrugbank2["category_code"] = lb_make.fit_transform(dfdrugbank2["category"])

        import pandas as pd

        dfdrugbank = pd.read_csv("C:/git/DMPrj/data/drugbank.tsv", delimiter="\t")
        dfprotein = pd.read_csv("C:/git/DMPrj/data/proteins.tsv", delimiter="\t")
        dfdrugbank['DrugBank_id'] = dfdrugbank['drugbank_id']
        dfprotein['DrugBank_id'] = dfprotein['drugbank_id']
        dfnodes = pd.read_csv("C:/git/DMPrj/data/DrugBank_DDI/node_list.txt", delimiter="	")
        dfnodes['ID'] = dfnodes.index
        dfnodes['NODEID'] = dfnodes['ID']
        dfnodes['WEIGHT'] = 0
        dfdrugbank2 = dfdrugbank.merge(dfnodes, how="left", on="DrugBank_id")
        dfdrugbank2 = dfdrugbank2.merge(dfprotein, how="left", on="DrugBank_id")
        # dfdrugbank3 = dfdrugbank2[str(dfdrugbank2['ID'])!="nan"]
        dfdrugbank2.dropna(inplace=True)
        from sklearn.preprocessing import LabelEncoder
        lb_make = LabelEncoder()
        dfdrugbank2["category_code"] = lb_make.fit_transform(dfdrugbank2["category"])
        dfdrugbank2["actions_code"] = lb_make.fit_transform(dfdrugbank2["actions"])
        dfdrugbank2["organism_code"] = lb_make.fit_transform(dfdrugbank2["organism"])
        dfdrugbank2["groups_code"] = lb_make.fit_transform(dfdrugbank2["groups"])

        dfdrugbank2['ID'] = dfdrugbank2['ID'].astype(int)
        self.dataset = dfdrugbank2
        #tc.dataset = dfdrugbank2
        #filename="C:/git/doc2vec/data/dataset.csv"
        #self.dataset = pd.read_csv(filename, header=0, delimiter="\t")


    def prepare_all_data(self,classifierparam):
        #x_train, x_test, y_train, y_test = train_test_split(
        #    self.dataset.review, self.dataset.sentiment, random_state=0,
        #    test_size=0.1)
        x_train, x_test, y_train, y_test = train_test_split(
            self.dataset.description, self.dataset[classifierparam], random_state=0,
            test_size=0.1)
        x_train = doc2VecModel.label_sentences(x_train, 'Train')
        x_test = doc2VecModel.label_sentences(x_test, 'Test')
        #x_train2 = doc2VecModel.label_sentences2(x_train, 'Train')
        all_data = x_train + x_test
        return x_train, x_test, y_train, y_test, all_data


    def prepare_test_data(self, sentence):
        x_test = doc2VecModel.label_sentences(sentence, 'Test')
        return x_test

    def train_classifier(self,classifierparam):
        x_train, x_test, y_train, y_test, all_data = self.prepare_all_data(classifierparam)
        self.d2v.initialize_model(all_data)
        self.d2v.train_model()
        self.classifier.initialize_model()
        self.classifier.train_model(self.d2v, x_train, y_train)
        self.classifier.test_model(self.d2v, x_test, y_test)
        return self.d2v, self.classifier

    def test_classifier(self,classifierparam):
        _, x_test, _, y_test, _ = self.prepare_all_data(classifierparam)
        if (self.d2v.model is None or self.classifier.model is None):
            logging.info(
                "Models Not Found, Train First or Use Correct Model Names")
        else:
            self.classifier.test_model(self.d2v, x_test, y_test)

In [35]:
tc = TextClassifier()
tc.read_data()

In [36]:
classifierparam="category_code"
tc.train_classifier(classifierparam="category_code")

2020-06-07 23:37:15,619 : INFO : Building Doc2Vec vocabulary
2020-06-07 23:37:15,622 : INFO : collecting all words and their counts
2020-06-07 23:37:15,622 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2020-06-07 23:37:15,762 : INFO : collected 8892 word types and 6807 unique tags from a corpus of 6807 examples and 385086 words
2020-06-07 23:37:15,763 : INFO : Loading a fresh vocabulary
2020-06-07 23:37:15,823 : INFO : effective_min_count=1 retains 8892 unique words (100% of original 8892, drops 0)
2020-06-07 23:37:15,827 : INFO : effective_min_count=1 leaves 385086 word corpus (100% of original 385086, drops 0)
2020-06-07 23:37:15,899 : INFO : deleting the raw counts dictionary of 8892 items
2020-06-07 23:37:15,901 : INFO : sample=0.001 downsamples 36 most-common words
2020-06-07 23:37:15,903 : INFO : downsampling leaves estimated 295887 word corpus (76.8% of prior 385086)
2020-06-07 23:37:15,947 : INFO : estimated required memory for 8892 words and 3

(<models.doc2vec_model.doc2VecModel at 0x1f47ed16978>,
 <models.classifier_model.classifierModel at 0x1f476ccd7f0>)

In [33]:
tc.test_classifier(classifierparam="category_code")




2020-06-07 23:36:03,375 : INFO : Classifier testing
2020-06-07 23:36:03,388 : INFO : Testing predicted classes: [1 2 3]
2020-06-07 23:36:03,391 : INFO : Testing accuracy: 0.4684287812041116
2020-06-07 23:36:03,399 : INFO : Testing F1 score: 0.4019787143647979
